In [1]:
import os,json
import numpy as np
from keras.models import Sequential
from keras.layers.core import Flatten, Dense, Dropout, Lambda
from keras.utils.data_utils import get_file
from keras.preprocessing import image
from keras.optimizers import SGD, RMSprop, Adam
from keras import backend as K
from keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D
from keras.layers import AveragePooling2D, MaxPooling2D, Dropout, GlobalMaxPooling2D, GlobalAveragePooling2D
from keras.models import Model
from keras.preprocessing import image
from keras.utils import layer_utils
import matplotlib.pyplot as plt
K.set_image_data_format('channels_last')
plt.style.use('ggplot')
%matplotlib inline

Using TensorFlow backend.


In [5]:
from keras.applications.resnet50 import ResNet50, preprocess_input
base_model = ResNet50(include_top=False, weights='imagenet', input_shape=(200,200,3))
base_model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_3 (InputLayer)             (None, 200, 200, 3)   0                                            
____________________________________________________________________________________________________
conv1 (Conv2D)                   (None, 100, 100, 64)  9472        input_3[0][0]                    
____________________________________________________________________________________________________
bn_conv1 (BatchNormalization)    (None, 100, 100, 64)  256         conv1[0][0]                      
____________________________________________________________________________________________________
activation_99 (Activation)       (None, 100, 100, 64)  0           bn_conv1[0][0]                   
___________________________________________________________________________________________

In [8]:
for layer in base_model.layers:
    layer.trainable =False
X = base_model.output
X = Flatten()(X)
X = Dense(activation='softmax', units=5)(X)
model = Model(inputs=base_model.input, outputs=X)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_3 (InputLayer)             (None, 200, 200, 3)   0                                            
____________________________________________________________________________________________________
conv1 (Conv2D)                   (None, 100, 100, 64)  9472        input_3[0][0]                    
____________________________________________________________________________________________________
bn_conv1 (BatchNormalization)    (None, 100, 100, 64)  256         conv1[0][0]                      
____________________________________________________________________________________________________
activation_99 (Activation)       (None, 100, 100, 64)  0           bn_conv1[0][0]                   
___________________________________________________________________________________________

In [13]:
path = 'data/flower_photos'
batches = image.ImageDataGenerator(preprocessing_function=preprocess_input).flow_from_directory(
    path, target_size=(200,200), class_mode='categorical', batch_size=16, shuffle=True)
assert batches.num_class==5
steps_per_epoch = batches.samples // batches.batch_size
model.fit_generator(
    batches, 
    steps_per_epoch=steps_per_epoch, 
    epochs=1)

Found 3670 images belonging to 5 classes.
Epoch 1/1
229/229 [==============================] - 1826s - loss: 0.3167 - acc: 0.8873  


In [2]:
def preprocess(x):
    MEAN = np.array([123.68, 116.779, 103.939]).reshape((1,1,3))
    x = x - MEAN
    # RGB -> BGR
    return x[:,:,::-1]

def ConvBlock(X, filters, k, s, stage, block):
    X_shortcut = X
    F1, F2, F3 = filters
    
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
        
    # main path
    X = Conv2D(filters=F1, kernel_size=(1,1), strides=(s,s), padding='valid', name=conv_name_base + '2a')(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2a')(X)
    X = Activation('relu')(X)
    
    X = Conv2D(filters=F2, kernel_size=(k,k), strides=(1,1), padding='same', name=conv_name_base + '2b')(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2b')(X)
    X = Activation('relu')(X)
    
    X = Conv2D(filters=F3, kernel_size=(1,1), strides=(1,1), padding='valid', name = conv_name_base + '2c')(X)
    X = BatchNormalization(axis=3, name = bn_name_base + '2c')(X)
        
    # shortcut path
    X_shortcut = Conv2D(filters=F3, kernel_size=(1,1), strides=(s,s), padding='valid', name = conv_name_base + '1')(X_shortcut)
    X_shortcut = BatchNormalization(axis=3, name = bn_name_base + '1')(X_shortcut)
    
    X = Add()([X_shortcut, X])
    X = Activation('relu')(X)
        
    return X

def IDBlock(X, filters, k, stage, block):
    X_shortcut = X
    F1, F2, F3 = filters
    
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    
    # mian path
    X = Conv2D(filters=F1, kernel_size=(1,1), strides=(1,1), padding='valid', name = conv_name_base + '2a')(X)
    X = BatchNormalization(axis=3, name = bn_name_base + '2a')(X)
    X = Activation('relu')(X)
    
    X = Conv2D(filters=F2, kernel_size=(k,k), strides=(1,1), padding='same', name = conv_name_base + '2b')(X)
    X = BatchNormalization(axis=3, name = bn_name_base + '2b')(X)
    X = Activation('relu')(X)
    
    X = Conv2D(filters=F3, kernel_size=(1,1), strides=(1,1), padding='valid', name = conv_name_base + '2c')(X)
    X = BatchNormalization(axis=3, name = bn_name_base + '2c')(X)
    
    # shortcut path
    X = Add()([X_shortcut, X])
    X = Activation('relu')(X)
    
    return X

In [3]:
def ResNet(input_shape):        
    X_input = Input(input_shape)
        
    # Stage 1
    X = Conv2D(filters=64, kernel_size=(7,7), strides=(2,2), padding='same', name = 'conv1')(X_input)
    X = BatchNormalization(axis=3, name = 'bn_conv1')(X)
    X = Activation('relu')(X)
    X = MaxPooling2D(pool_size=(3, 3), strides=(2,2), padding='valid')(X)
        
    # Stage 2
    X = ConvBlock(X, filters=[64,64,256], k=3, s=1, stage = 2, block='a')    
    X = IDBlock(X, filters=[64,64,256], k=3, stage=2, block='b')
    X = IDBlock(X, filters=[64,64,256], k=3, stage=2, block='c')
        
    # Stage 3
    X = ConvBlock(X, filters=[128,128,512], k=3, s=2, stage = 3, block='a')
    X = IDBlock(X, filters=[128,128,512], k=3, stage=3, block='b')
    X = IDBlock(X, filters=[128,128,512], k=3, stage=3, block='c')
    X = IDBlock(X, filters=[128,128,512], k=3, stage=3, block='d')
        
    # Stage 4
    X = ConvBlock(X, filters=[256,256,1024], k=3, s=2, stage = 4, block='a')
    X = IDBlock(X, filters=[256,256,1024], k=3, stage=4, block='b')
    X = IDBlock(X, filters=[256,256,1024], k=3, stage=4, block='c')
    X = IDBlock(X, filters=[256,256,1024], k=3, stage=4, block='d')
    X = IDBlock(X, filters=[256,256,1024], k=3, stage=4, block='e')
    X = IDBlock(X, filters=[256,256,1024], k=3, stage=4, block='f')
        
    # Stage 5
    X = ConvBlock(X, filters=[512,512,2048], k=3, s=2, stage = 5, block='a')
    X = IDBlock(X, filters=[512,512,2048], k=3, stage=5, block='b')
    X = IDBlock(X, filters=[512,512,2048], k=3, stage=5, block='c')
        
    X = AveragePooling2D(pool_size=(7,7), padding='valid', name = 'avg_pool')(X)
    
    model = Model(inputs=X_input, outputs=X)
    return model

In [4]:
base_model1 = ResNet((200,200,3))
base_model1.load_weights('data/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5', by_name=True)
for layer in base_model1.layers:
    layer.trainable =False
X = base_model1.output
X = Flatten()(X)
X = Dense(activation='softmax', units=5)(X)
model1 = Model(inputs=base_model1.input, outputs=X)
model1.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [6]:
path = 'data/flower_photos'
batches = image.ImageDataGenerator(preprocessing_function=preprocess).flow_from_directory(
    path, target_size=(200,200), class_mode='categorical', batch_size=16, shuffle=True)
assert batches.num_class==5
steps_per_epoch = batches.samples // batches.batch_size
model1.fit_generator(
    batches, 
    steps_per_epoch=steps_per_epoch, 
    epochs=1)

Found 3670 images belonging to 5 classes.
Epoch 1/1
229/229 [==============================] - 1820s - loss: 0.5876 - acc: 0.7789  
